In [1]:
import datetime
import praw
import pandas as pd

from keys import client_id, secret

### Collecting posts

In [2]:
reddit = praw.Reddit(client_id=client_id, client_secret=secret, user_agent='postscrape by /u/kejimuna')

In [3]:
tesla = reddit.subreddit('TeslaMotors')

posts = []
for index, post in enumerate(tesla.top(limit=500)):
    posts.append([post.title, "https://reddit.com" + post.permalink, post.selftext, post.score, post.created_utc, index])

# dataframe for posts
posts = pd.DataFrame(posts, columns=['Title', 'URL', 'Body', 'Upvotes', 'Time', 'Key'])

# changing from utc time to standard timestamp
posts.Time = posts.Time.apply(lambda x: pd.to_datetime(datetime.datetime.fromtimestamp(x)))

# drop first post as its a sticky
posts = posts.iloc[1:]

In [4]:
posts.head(5)

,Title,URL,Body,Upvotes,Time,Key
1,Elon Musk confirms secret Tesla ‘Carless Drive...,https://reddit.com/r/teslamotors/comments/7tvn...,,45604,2018-01-30 10:20:51,1
2,Tesla vs Bugatti,https://reddit.com/r/teslamotors/comments/7e2z...,,44270,2017-11-20 08:53:56,2
3,Autopilot saves my model 3 from an accident!,https://reddit.com/r/teslamotors/comments/andk...,,39400,2019-02-06 01:20:41,3
4,Update to the previous post,https://reddit.com/r/teslamotors/comments/7p6z...,,33411,2018-01-10 02:09:52,4
5,DAY ONE WITH MY TESLA!,https://reddit.com/r/teslamotors/comments/7yjr...,,32047,2018-02-19 16:27:47,5


In [5]:
posts.shape

(499, 6)

Collecting the comments

In [6]:
def collect_replies(key, url):
    '''
    Args:
        key: the key of the post
        url (str): the url of the post

    Returns:
        pd.dataframe: comments
    '''

    submission = reddit.submission(url=url)
    submission.comments.replace_more(limit=None)
    comment_queue = submission.comments[:]

    table = {'Reply':[], 'Upvote':[], 'Time':[], 'Key':[]}

    while comment_queue:
        comment = comment_queue.pop(0)
        table['Reply'].append(comment.body)
        table['Time'].append(comment.created_utc)
        table['Upvote'].append(comment.score)
        table['Key'].append(key)
        comment_queue.extend(comment.replies)

    return pd.DataFrame.from_dict(table)

In [7]:
keys = posts.Key.to_list()
urls = posts.URL.to_list()
tuples = list(zip(keys, urls))

comments = pd.concat([collect_replies(x[0], x[1]) for x in tuples])

KeyboardInterrupt: 

In [ ]:
comments.Time = comments.Time.apply(lambda x: pd.to_datetime(datetime.datetime.fromtimestamp(x)))

In [ ]:
comments.shape